https://github.com/ArjanCodes/examples/

### Nested list comprehension

In [1]:
lists = [[1,2,3,4],[5,6,7,8]]

In [3]:
[r for r in lists]

[[1, 2, 3, 4], [5, 6, 7, 8]]

nested comprehension

In [4]:
[num for row in lists for num in row]

[1, 2, 3, 4, 5, 6, 7, 8]

### F-string formatting

In [7]:
pi = 3.142454
print(f'Pi to 2 sf: {pi:.2f}')

Pi to 2 sf: 3.14


In [3]:
pop = 6123456
print(f'Population of Singapore: {pop:,} people')

Population of Singapore: 6,123,456 people


In [7]:
pop = 0.492364
print(f'Percentage of males: {pop:.3%} of people')

Percentage of males: 49.236% of people


In [14]:
from datetime import datetime

now = datetime.now()
print(f'datetime now: {now:%Y-%m-%d}')

datetime now: 2025-01-28


alignment

In [18]:
name = 'Python'
print(f'{name:<20} is left aligned')
print(f'{name:^20} is centered')
print(f'{name:>20} is right aligned')

Python               is left aligned
       Python        is centered
              Python is right aligned


debugging

In [20]:
val = 42
print(f'{val=}')
print(f'{val**2=}')

val=42
val**2=1764


zip for matrix rotation

In [21]:
def rotate_matrix_clockwise(matrix):
    return [list(row) for row in zip(*matrix[::-1])]

# Example usage
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

rotated = rotate_matrix_clockwise(matrix)
for row in rotated:
    print(row)


[7, 4, 1]
[8, 5, 2]
[9, 6, 3]


### Context Manager

In [26]:
import time
from contextlib import contextmanager

@contextmanager
def timer():
    start = time.time()
    yield 
    # with func runs until here
    end = time.time()
    print(f'Elapsed time : {end - start:.2f} seconds')
    # code until here runs after exiting context management

with timer():
    total = sum(range(10_000_000))
    print(f'Sum computed: {total}')

Sum computed: 49999995000000
Elapsed time : 0.23 seconds


### Typing

In [ ]:
from typing import Iterable, Optional

### Abstract Base Classes

In [27]:
from abc import ABC, abstractmethod


class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount: float) -> None:
        pass


class PayPalProcessor(PaymentProcessor):
    def process_payment(self, amount: float) -> None:
        print(f"Processing ${amount} payment via PayPal.")


class StripeProcessor(PaymentProcessor):
    def process_payment(self, amount: float) -> None:
        print(f"Processing ${amount} payment via Stripe.")


def process_order(amount: float, processor: PaymentProcessor) -> None:
    processor.process_payment(amount)


paypal = PayPalProcessor()
stripe = StripeProcessor()
process_order(100.0, paypal)
process_order(200.0, stripe)

Processing $100.0 payment via PayPal.
Processing $200.0 payment via Stripe.


### Classes, Dataclasses and Functions

In [28]:
from decimal import Decimal


class ShoppingCart:
    def __init__(self) -> None:
        self.items = []

    def add_item(self, item: str, price: Decimal) -> None:
        self.items.append({"item": item, "price": price})

    def total(self) -> Decimal:
        return sum(item["price"] for item in self.items)


cart = ShoppingCart()
cart.add_item("apple", Decimal("1.5"))
cart.add_item("banana", Decimal("2.0"))
print(f"${cart.total():.2f}")

$3.50


In [29]:
from dataclasses import dataclass, field
from decimal import Decimal


@dataclass # order enables sorting, frozen locks editing
# @dataclass(order = True, frozen = True)
class Item:
    name: str
    price: Decimal
    # sort_index: int = field(init = False, repr = False)
    # strength: int = 100

    # def __post_init__(self):
    #   self.sort_index = self.strength
    #   object.__setattr__(self, 'sort_index', self.strength)


item = Item(name="apple", price=Decimal("1.5"))
print(item)
print(f"${item.price:.2f}")

Item(name='apple', price=Decimal('1.5'))
$1.50


functions for stateless operations

In [30]:
from dataclasses import dataclass
from decimal import Decimal


@dataclass
class Item:
    name: str
    price: Decimal


class ShoppingCart:
    def __init__(self) -> None:
        self.items: list[Item] = []

    def add_item(self, item: Item) -> None:
        self.items.append(item)

    def total(self) -> Decimal:
        return sum(item.price for item in self.items)


def calculate_discount(cart: ShoppingCart, discount: Decimal) -> Decimal:
    return cart.total() * (1 - discount / 100)


cart = ShoppingCart()
cart.add_item(Item("apple", Decimal("1.5")))
cart.add_item(Item("banana", Decimal("2.0")))

discounted_price = calculate_discount(cart, Decimal("10"))
print(f"${discounted_price:.2f}")

$3.15


### Decorators

https://github.com/ArjanCodes/2023-decorator/blob/main/decorator_func.py

In [ ]:
import logging
from math import sqrt
from time import perf_counter
from typing import Any, Callable


def with_logging(func: Callable[..., Any]) -> Callable[..., Any]:
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        logging.info(f"Calling {func.__name__}")
        value = func(*args, **kwargs)
        logging.info(f"Finished {func.__name__}")
        return value

    return wrapper


def benchmark(func: Callable[..., Any]) -> Callable[..., Any]:
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start_time = perf_counter()
        value = func(*args, **kwargs)
        end_time = perf_counter()
        run_time = end_time - start_time
        logging.info(f"Execution of {func.__name__} took {run_time:.2f} seconds.")
        return value

    return wrapper


def is_prime(number: int) -> bool:
    if number < 2:
        return False
    for element in range(2, sqrt(number) + 1):
        if number % element == 0:
            return False
    return True


@with_logging
@benchmark
def count_prime_numbers(upper_bound: int) -> int:
    count = 0
    for number in range(upper_bound):
        if is_prime(number):
            count += 1
    return count


def main() -> None:
    logging.basicConfig(level=logging.INFO)
    count_prime_numbers(50000)


if __name__ == "__main__":
    main()

### Partial (functools)

https://github.com/ArjanCodes/2022-functions/blob/main/strategy_fn_partial.py

In [31]:
"""
Basic example of a Trading bot with a strategy pattern.
"""
import statistics
from dataclasses import dataclass
from functools import partial
from typing import Callable

from exchange import Exchange

TradingStrategyFunction = Callable[[list[int]], bool]


def should_buy_avg(prices: list[int], window_size: int) -> bool:
    list_window = prices[-window_size:]
    return prices[-1] < statistics.mean(list_window)


def should_sell_avg(prices: list[int], window_size: int) -> bool:
    list_window = prices[-window_size:]
    return prices[-1] > statistics.mean(list_window)


def should_buy_minmax(prices: list[int], max_price: int) -> bool:
    # buy if it's below the max price
    return prices[-1] < max_price


def should_sell_minmax(prices: list[int], min_price: int) -> bool:
    # sell if it's above the min price
    return prices[-1] > min_price


@dataclass
class TradingBot:
    """Trading bot that connects to a crypto exchange and performs trades."""

    exchange: Exchange
    buy_strategy: TradingStrategyFunction
    sell_strategy: TradingStrategyFunction

    def run(self, symbol: str) -> None:
        prices = self.exchange.get_market_data(symbol)
        if self.buy_strategy(prices):
            self.exchange.buy(symbol, 10)
        elif self.sell_strategy(prices):
            self.exchange.sell(symbol, 10)
        else:
            print(f"No action needed for {symbol}.")


def main() -> None:
    # create the exchange and connect to it
    exchange = Exchange()
    exchange.connect()

    # create the trading bot and run the bot once
    buy_strategy = partial(should_buy_minmax, max_price=32_000_00)
    sell_strategy = partial(should_sell_minmax, min_price=38_000_00)
    bot = TradingBot(exchange, buy_strategy, sell_strategy)
    bot.run("BTC/USD")


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'exchange'

### Pydantic

https://github.com/ArjanCodes/2021-pydantic/blob/main/example.py

In [ ]:
"""
Basic example showing how to read and validate data from a file using Pydantic.
"""

import json
from typing import List, Optional

import pydantic


class ISBNMissingError(Exception):
    """Custom error that is raised when both ISBN10 and ISBN13 are missing."""

    def __init__(self, title: str, message: str) -> None:
        self.title = title
        self.message = message
        super().__init__(message)


class ISBN10FormatError(Exception):
    """Custom error that is raised when ISBN10 doesn't have the right format."""

    def __init__(self, value: str, message: str) -> None:
        self.value = value
        self.message = message
        super().__init__(message)


class Author(pydantic.BaseModel):
    name: str
    verified: bool


class Book(pydantic.BaseModel):
    """Represents a book with that you can read from a JSON file."""

    title: str
    author: str
    publisher: str
    price: float
    isbn_10: Optional[str]
    isbn_13: Optional[str]
    subtitle: Optional[str]
    author2: Optional[Author]

    @pydantic.root_validator(pre=True)
    @classmethod
    def check_isbn_10_or_13(cls, values):
        """Make sure there is either an isbn_10 or isbn_13 value defined"""
        if "isbn_10" not in values and "isbn_13" not in values:
            raise ISBNMissingError(
                title=values["title"],
                message="Document should have either an ISBN10 or ISBN13",
            )
        return values

    @pydantic.validator("isbn_10")
    @classmethod
    def isbn_10_valid(cls, value) -> None:
        """Validator to check whether ISBN10 is valid"""
        chars = [c for c in value if c in "0123456789Xx"]
        if len(chars) != 10:
            raise ISBN10FormatError(value=value, message="ISBN10 should be 10 digits.")

        def char_to_int(char: str) -> int:
            if char in "Xx":
                return 10
            return int(char)

        if sum((10 - i) * char_to_int(x) for i, x in enumerate(chars)) % 11 != 0:
            raise ISBN10FormatError(
                value=value, message="ISBN10 digit sum should be divisible by 11."
            )
        return value

    class Config:
        """Pydantic config class"""

        allow_mutation = False
        anystr_lower = True


def main() -> None:
    """Main function."""

    # Read data from a JSON file
    with open("./data.json") as file:
        data = json.load(file)
        books: List[Book] = [Book(**item) for item in data]
        # print(books)
        print(books[0])
        # print(books[0].dict(exclude={"price"}))
        # print(books[1].copy())


if __name__ == "__main__":
    main()